In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-insincere-questions-classification/sample_submission.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/test.csv
/kaggle/input/insincere_question_final_model/pytorch/1.01/1/insincere_model_final_successful.pth
/kaggle/input/bert_tokenizer/pytorch/1.01/1/Bert Tokenizer.pth
/kaggle/input/bert-base-uncased/pytorch/1.01/1/BertModel_base_uncased.pth


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import RobertaTokenizer, RobertaModel
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tqdm import tqdm

from sklearn.utils.class_weight import compute_class_weight
import torch.utils
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import MultiStepLR

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
df_train = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/train.csv")
df_test = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/test.csv")
sub_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/sample_submission.csv')

In [5]:
class TrainDataset(Dataset):
    def __init__(self, question_text,targets, tokenizer, max_length=60):
        self.texts = question_text
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target = float(self.targets[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            add_special_tokens=True,
            return_tensors='pt'
        )
        return (encoding['input_ids'].flatten(), encoding['attention_mask'].flatten(),torch.tensor(target))

In [6]:
class TestDataset(Dataset):
    def __init__(self,df_test, tokenizer, max_length = 60):
        self.texts = df_test.question_text
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer.encode_plus(
        text,
        max_length = self.max_length,
        padding = 'max_length',
        truncation = True,
        add_special_tokens = True,
        return_tensors = 'pt'
        )
        return (encoding['input_ids'].flatten(),encoding['attention_mask'].flatten())

In [7]:
bert_tokenizer = torch.load('/kaggle/input/bert_tokenizer/pytorch/1.01/1/Bert Tokenizer.pth')

In [8]:
# Load pre-trained BERT tokenizer
# bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
# train_inputs, val_inputs,train_targets, val_targets = train_test_split(df_train.question_text, df_train.target,stratify=df_train.target,test_size=0.2, random_state=42)

In [10]:
# train_inputs.reset_index(drop=True,inplace=True)
# train_targets.reset_index(drop=True,inplace=True)
# val_targets.reset_index(drop=True,inplace=True)
# val_inputs.reset_index(drop=True,inplace=True)

In [11]:
# train_ds = TrainDataset(train_inputs,train_targets,bert_tokenizer)
# val_ds = TrainDataset(val_inputs,val_targets,bert_tokenizer)
# train_dl = DataLoader(train_ds,  batch_size=256)
# val_dl = DataLoader(val_ds, batch_size=512)

In [12]:
# class BERT_MODEL(nn.Module):
#     def __init__(self):
#         super(BERT_MODEL,self).__init__()
#         self.bert_model = BertModel.from_pretrained('bert-base-uncased')
#         self.linear = nn.Sequential(
#         nn.Linear(768,1024),
#         nn.ReLU(),
#         nn.Dropout(0.3),
#         nn.Linear(1024,1)
#         )
    
#     def forward(self,input_ids,attention_mask):   
#         output = self.bert_model(input_ids = input_ids,attention_mask = attention_mask)
#         pooled_output = output.pooler_output
#         output = self.linear(pooled_output)
#         return output

In [8]:
class BERT_MODEL(nn.Module):
    def __init__(self):
        super(BERT_MODEL,self).__init__()
        self.bert_model = torch.load('/kaggle/input/bert-base-uncased/pytorch/1.01/1/BertModel_base_uncased.pth')
        self.linear = nn.Sequential(
        nn.Linear(768,1024),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(1024,1)
        )
    
    def forward(self,input_ids,attention_mask):   
        output = self.bert_model(input_ids = input_ids,attention_mask = attention_mask)
        pooled_output = output.pooler_output
        output = self.linear(pooled_output)
        return output

In [14]:
# model = BERT_MODEL()

In [15]:
# model.to(device)

In [16]:
# def find_best_f1(outputs, labels):
#     tmp = [0, 0, 0]  # idx, cur, max
#     threshold = 0

#     for tmp[0] in np.arange(0.1, 0.99, 0.01):
#         tmp[1] = f1_score(labels, outputs > tmp[0])
#         if tmp[1] > tmp[2]:
#             threshold = tmp[0]
#             tmp[2] = tmp[1]

#     return tmp[2], threshold

In [17]:
# def get_preds(logits,threshold):
    
#     # Convert logits to binary predictions based on the threshold
#     predictions = (torch.sigmoid(logits) > threshold).float()
    
#     return predictions

In [19]:
# def evaluate(model,val_dl):
#     losses=[]
#     val_outputs = []
#     val_targets = []
#     accuracy = []
#     f1 = []
#     i=1
#     model.eval()
#     with torch.no_grad():
#         for batch in tqdm(val_dl):
#             input_ids,attention_mask, targets = batch
#             input_ids = input_ids.to(device)
#             attention_mask = attention_mask.to(device)
#             targets = targets.to(device)## target is of type 0.0 and 1.0
            
#             output = model(input_ids,attention_mask)
            
#             loss = BCE(output.squeeze(), targets.float())
            
#             val_outputs.append(torch.sigmoid(output).squeeze().cpu().numpy())
#             val_targets.append(targets.cpu().numpy())
# #             losses.append(loss.item())
# #             f1_Score = f1_score(output.cpu().numpy(),targets.cpu().numpy().astype(int))
# #             f1.append(f1_Score)
# #             if i==5:
# #                 return  val_outputs,val_targets
# #                 break
# #             i+=1
# #             print(f'Val_loss: {loss},val_f1_Score:{f1_Score}')
#     return  val_outputs,val_targets

In [20]:
# BCE = nn.BCEWithLogitsLoss(pos_weight = torch.tensor(15,device=device))


In [27]:
# def fit(epochs,model,train_dl,val_dl):
#     optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4)
#     history = []
#     losses = []
#     train_outputs = []
#     i = 1
#     milestones = [1500, 3000]
#     scheduler = MultiStepLR(optimizer, milestones= milestones, gamma=0.1)
#     model.train()
#     for epoch in range(epochs):
#         for batch in tqdm(train_dl):
#             input_ids, attention_mask, targets = batch
            
#             input_ids = input_ids.to(device)
#             attention_mask = attention_mask.to(device)
#             targets = targets.to(device) 
            
#             output = model(input_ids, attention_mask)
            
#             loss = BCE(output.squeeze(),targets)
#             losses.append(loss.item())
            
#             optimizer.zero_grad()
#             loss.backward()
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5000)
#             optimizer.step()
#             scheduler.step()
            
#             if (i % 10 == 0):
# #                 for name, param in model.named_parameters():
# #                     if (param.grad is not None) & (param.grad.abs().sum()>5000):
# #                         print(name, param.grad.abs().sum())
# #                         torch.save(model.state_dict(), 'insincere_model.pth')
#                 f1, thres = find_best_f1(torch.sigmoid(output.detach()).squeeze().cpu().numpy(), targets.cpu().numpy())
#                 print(f'Batch:{i} ; Loss: {loss:.3f}; Pred at {thres:.3}:{get_preds(output.squeeze(),thres)}; best_Train_f1:{f1:.3f}')
                
#             if (i % 10 == 0):
#                 current_lr = optimizer.param_groups[0]['lr']
#                 print("\nCurrent learning rate:", current_lr)
#                 torch.save(model.state_dict(), 'insincere_model.pth')
#             i += 1
         
#         val_outputs,val_targets = evaluate(model,val_dl)
#         val_outputs = np.concatenate(val_outputs)
#         val_targets = np.concatenate(val_targets)
#         val_f1, threshold = find_best_f1(val_outputs, val_targets)
#         print("Epoch {}; Val F1: {:.3f}, Threshold: {:.3f}".format(epoch,val_f1, threshold))
#     return [val_f1,threshold,val_outputs,val_targets]

In [41]:
# results = fit(1,model,train_dl,val_dl) Hyper parameter tuning, getting bad results

  0%|          | 10/8164 [00:07<1:52:02,  1.21it/s]

Batch:10 ; Loss: 0.762; Pred at 0.7699999999999997:tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], device='cuda:0'); best_Train_f1:0.545


  0%|          | 20/8164 [00:15<1:52:24,  1.21it/s]

Batch:20 ; Loss: 1.034; Pred at 0.5099999999999998:tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
        0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0.], device='cuda:0'); best_Train_f1:0.541


  0%|          | 30/8164 [00:23<1:52:30,  1.21it/s]

...

Batch:3990 ; Loss: 0.948; Pred at 0.5399999999999998:tensor([1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], device='cuda:0'); best_Train_f1:0.100


 49%|████▉     | 3999/8164 [53:26<52:59,  1.31it/s]

Batch:4000 ; Loss: 1.251; Pred at 0.1:tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], device='cuda:0'); best_Train_f1:0.104


 49%|████▉     | 4002/8164 [53:31<55:39,  1.25it/s]  


KeyboardInterrupt: 

In [22]:
# results = fit(1,model,train_dl,val_dl) #### Hyper parameter tuning (again getting bad resuts)

  0%|          | 10/8164 [00:07<1:56:11,  1.17it/s]

Batch:10 ; Loss: 0.747; Pred at 0.71:tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1.], device='cuda:0'); best_Train_f1:0.526


  0%|          | 20/8164 [00:14<1:48:16,  1.25it/s]

Batch:20 ; Loss: 0.868; Pred at 0.66:tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0.], device='cuda:0'); best_Train_f1:0.645


  0%|          | 30/8164 [00:21<1:48:50,  1.25it/s]

...

Batch:5240 ; Loss: 1.533; Pred at 0.1:tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], device='cuda:0'); best_Train_f1:0.158


 64%|██████▍   | 5250/8164 [58:30<39:23,  1.23it/s]

Batch:5250 ; Loss: 1.452; Pred at 0.1:tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], device='cuda:0'); best_Train_f1:0.145


 64%|██████▍   | 5255/8164 [58:33<32:25,  1.50it/s]


KeyboardInterrupt: 

In [32]:
# results = []

In [33]:
# results.append(fit(5,model,train_dl,val_dl))

  0%|          | 9/4082 [00:10<1:23:46,  1.23s/it]

Batch:10 ; Loss: 0.944; Pred at 0.76:tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
   

  0%|          | 19/4082 [00:24<1:25:44,  1.27s/it]

Batch:20 ; Loss: 0.857; Pred at 0.47:tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
   

  1%|          | 29/4082 [00:38<1:25:31,  1.27s/it]

...

Batch:8160 ; Loss: 0.527; Pred at 0.79:tensor([0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
 

100%|██████████| 511/511 [10:31<00:00,  1.24s/it]


Epoch 1; Val F1: 0.700, Threshold: 0.900


  0%|          | 3/4082 [00:04<1:39:17,  1.46s/it]


KeyboardInterrupt: 

In [34]:
# torch.save(model.state_dict(), 'insincere_model_final_successful.pth')

In [36]:
# results = evaluate(model,val_dl)

100%|██████████| 511/511 [10:31<00:00,  1.24s/it]


In [40]:
# val_outputs,val_targets = results
# val_outputs = np.concatenate(val_outputs)
# val_targets = np.concatenate(val_targets)
# val_f1, threshold = find_best_f1(val_outputs, val_targets)
# print("Val F1: {:.3f} at Threshold: {:.3f}".format(val_f1, threshold))

Val F1: 0.701 at Threshold: 0.900


In [15]:
best_threshold = 0.9

In [77]:
# encoding = bert_tokenizer.encode_plus(
#             "Why are IITs so bad at research?",
#             max_length=60,
#             padding='max_length',
#             truncation=True,
#             add_special_tokens=True,
#             return_tensors='pt'
#         ) 

In [80]:
# output = model(encoding['input_ids'].to(device), encoding['attention_mask'].to(device))

In [81]:
# get_preds(output.squeeze(),0.9)

tensor(0., device='cuda:0')

In [10]:
# del model

NameError: name 'model' is not defined

In [9]:
model = BERT_MODEL()
# model.to(device)

In [10]:
model.load_state_dict(torch.load('/kaggle/input/insincere_question_final_model/pytorch/1.01/1/insincere_model_final_successful.pth'))

<All keys matched successfully>

In [21]:
def test(df_test,model):
    test_dataset = TestDataset(df_test,tokenizer = bert_tokenizer)
    test_dl = DataLoader(test_dataset, batch_size = 32)
    preds = []
    for batch in tqdm(test_dl):
        input_ids , attention_mask = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        output = model(input_ids, attention_mask)
        del input_ids,attention_mask
        prediction = (torch.sigmoid(output).squeeze() > best_threshold).int()
        preds.append(prediction.cpu().squeeze().int().numpy())
    print('test predictions generated successfully!!')    
    return preds

In [35]:
def submission(sub_df,df_test,model):
    preds = test(df_test,model)
    predictions = np.array([])
    for pred in preds:
       predictions = np.concatenate([predictions,np.array(pred)])
    sub_df.prediction = predictions
    print('Submission Dataframe created successfully')
    return sub_df

In [13]:
model.to(device)

BERT_MODEL(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [49]:
submission_df = submission(sub_df,df_test,model)
submission_df.prediction = submission_df.prediction.astype(int)
submission_df.to_csv('submission.csv', index = False)

  0%|          | 37/11744 [00:02<15:08, 12.88it/s]


KeyboardInterrupt: 